In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
from scipy.stats import norm

In [3]:
tickers=['JPM','MS','BAC']

In [4]:
stock_data = yf.download(tickers, start='2022-8-1', end='2023-8-1', auto_adjust=True, interval='1d')

[*********************100%%**********************]  3 of 3 completed


In [5]:
stock_data.head()

Price           Close                              High              \
Ticker            BAC         JPM         MS        BAC         JPM   
Date                                                                  
2022-08-01  31.869627  108.014946  78.671326  32.039800  108.866129   
2022-08-02  31.236198  106.331490  78.147415  31.879074  108.194633   
2022-08-03  31.803442  107.447487  79.569473  31.907437  107.910906   
2022-08-04  31.576548  106.265297  79.859489  31.831804  107.173223   
2022-08-05  32.105968  109.480873  80.579880  32.455771  109.887549   

Price                        Low                              Open  \
Ticker             MS        BAC         JPM         MS        BAC   
Date                                                                 
2022-08-01  79.092332  31.406380  106.936783  77.660925  31.784540   
2022-08-02  79.139108  31.198381  106.180165  77.614146  31.765623   
2022-08-03  80.074670  31.359100  105.650547  78.699401  31.595452   
2022-08-04  80.130803  31.359101  104.997978  79.335578  31.689995   
2022-08-05  81.122507  31.425279  105.972114  79.251385  31.463092   

Price                                Volume                     
Ticker             JPM         MS       BAC       JPM       MS  
Date                                                            
2022-08-01  108.289216  78.269041  32336700  14740400  5439200  
2022-08-02  107.740666  77.960306  37884100  14942900  7684700  
2022-08-03  107.296162  79.223313  30065800  14394500  7043600  
2022-08-04  106.728716  79.709796  28404500  18033900  5722600  
2022-08-05  106.426074  79.513341  35874500  14174600  6219300

In [6]:
df_backup=stock_data['Close']
df=df_backup.copy()

In [7]:
df.head()

Ticker,BAC,JPM,MS
Date,,,
2022-08-01,31.869627,108.014946,78.671326
2022-08-02,31.236198,106.331490,78.147415
2022-08-03,31.803442,107.447487,79.569473
2022-08-04,31.576548,106.265297,79.859489
2022-08-05,32.105968,109.480873,80.579880


In [8]:
df['BAC_returns']=df['BAC'].pct_change()
df.head()

Ticker,BAC,JPM,MS,BAC_returns
Date,,,,
2022-08-01,31.869627,108.014946,78.671326,NaN
2022-08-02,31.236198,106.331490,78.147415,-0.019876
2022-08-03,31.803442,107.447487,79.569473,0.018160
2022-08-04,31.576548,106.265297,79.859489,-0.007134
2022-08-05,32.105968,109.480873,80.579880,0.016766


In [9]:
df['JPM_returns']=df['JPM'].pct_change()
df['MS_returns']=df['MS'].pct_change()
df.head()

Ticker,BAC,JPM,MS,BAC_returns,JPM_returns,MS_returns
Date,,,,,,
2022-08-01,31.869627,108.014946,78.671326,NaN,NaN,NaN
2022-08-02,31.236198,106.331490,78.147415,-0.019876,-0.015585,-0.006659
2022-08-03,31.803442,107.447487,79.569473,0.018160,0.010495,0.018197
2022-08-04,31.576548,106.265297,79.859489,-0.007134,-0.011002,0.003645
2022-08-05,32.105968,109.480873,80.579880,0.016766,0.030260,0.009021


In [10]:
portfolio_value=1_000_000
w_bac=0.3
w_jpm=0.3
w_ms=0.4
weights=np.array([w_bac,w_jpm,w_ms])

In [11]:
df['returns']=(w_bac*df['BAC_returns'])+(w_jpm*df['JPM_returns'])+(w_ms*df['MS_returns'])
df.head()

Ticker,BAC,JPM,MS,BAC_returns,JPM_returns,MS_returns,returns
Date,,,,,,,
2022-08-01,31.869627,108.014946,78.671326,NaN,NaN,NaN,NaN
2022-08-02,31.236198,106.331490,78.147415,-0.019876,-0.015585,-0.006659,-0.013302
2022-08-03,31.803442,107.447487,79.569473,0.018160,0.010495,0.018197,0.015875
2022-08-04,31.576548,106.265297,79.859489,-0.007134,-0.011002,0.003645,-0.003983
2022-08-05,32.105968,109.480873,80.579880,0.016766,0.030260,0.009021,0.017716


In [12]:
portfolio_mean_return=df['returns'].mean()
portfolio_mean_return

0.000737082379248206

In [13]:
cov_matrix=df[['BAC_returns','JPM_returns','MS_returns']].dropna().cov()
cov_matrix

Ticker,BAC_returns,JPM_returns,MS_returns
Ticker,,,
BAC_returns,0.000333,0.000243,0.000236
JPM_returns,0.000243,0.000265,0.000186
MS_returns,0.000236,0.000186,0.000296


In [14]:
portfolio_var=np.dot(weights.T, np.dot(cov_matrix, weights))
portfolio_var

0.0002462210363773998

In [15]:
portfolio_std=np.sqrt(portfolio_var)
portfolio_std

0.015691431941585184

In [16]:
ci_99=0.99
alpha_99=1-ci_99
z_99=norm.ppf(1-(alpha_99/2))
oned_ci_99=(portfolio_mean_return + (z_99*portfolio_std))*portfolio_value
print('One day VaR for a 99% CI is:', oned_ci_99)

One day VaR for a 99% CI is: 41155.532589026545


In [17]:
ci=0.95
alpha=1-ci
z=norm.ppf(1-(alpha/2))
oned_ci=(portfolio_mean_return + (z*portfolio_std))*portfolio_value
print(f'One day VaR for a {ci*100}% CI is:', oned_ci)

One day VaR for a 95.0% CI is: 31491.723850616585


In [18]:
ci=0.90
alpha=1-ci
z=norm.ppf(1-(alpha/2))
oned_ci=(portfolio_mean_return + (z*portfolio_std))*portfolio_value
print(f'One day VaR for a {ci*100}% CI is:', oned_ci)

One day VaR for a 90.0% CI is: 26547.191120426778


# Historical method

In [20]:
returns_list=df['returns'].dropna().to_list()

In [21]:
trading_days_drop=len(returns_list)

In [22]:
returns_list.sort()

In [23]:
ci=0.99
alpha=1-ci
rank=int(np.ceil(trading_days_drop*alpha))
ret_var=returns_list[rank-1]
var_hm= portfolio_value*ret_var
print(f'One day VaR uisng Historical method for a {ci*100}% CI is:', var_hm)

One day VaR uisng Historical method for a 99.0% CI is: -36253.49286862141


# MC method

In [26]:
def var_mc(S0, mu, sigma, T, nsims, ci):
    prices = []
    for _ in range(nsims):
        price_t = S0* np.exp((mu - 0.5 * sigma ** 2)*T + sigma * np.random.normal()* np.sqrt(T))
        prices.append(price_t)
    prices = np.sort(prices)
    price_ci = np.percentile(prices, ci*100)

    var = S0 - price_ci
    return var

In [27]:
var_mc(portfolio_value, 0.0007,0.0002, 1, 100_000, 0.95)

-1027.8889514931943